In [ ]:
import torch

In [ ]:
# datasets = torch.load("/content/drive/MyDrive/Pro2/databert")

In [ ]:
import torch
import torch.nn as nn
!pip install pytorch_pretrained_bert
from pytorch_pretrained_bert import BertModel, BertConfig

     |████████████████████████████████| 133kB 4.9MB/s 
     |████████████████████████████████| 133kB 37.7MB/s 
     |████████████████████████████████| 81kB 9.2MB/s 
     |████████████████████████████████| 7.6MB 36.4MB/s 
ERROR: botocore 1.20.87 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl

In [ ]:
""" Optimizers class """
import torch
import torch.optim as optim
from torch.nn.utils import clip_grad_norm_


# from onmt.utils import use_gpu


def use_gpu(opt):
    """
    Creates a boolean if gpu used
    """
    return (hasattr(opt, 'gpu_ranks') and len(opt.gpu_ranks) > 0) or \
           (hasattr(opt, 'gpu') and opt.gpu > -1)


class MultipleOptimizer(object):
    """ Implement multiple optimizers needed for sparse adam """

    def __init__(self, op):
        """ ? """
        self.optimizers = op

    def zero_grad(self):
        """ ? """
        for op in self.optimizers:
            op.zero_grad()

    def step(self):
        """ ? """
        for op in self.optimizers:
            op.step()

    @property
    def state(self):
        """ ? """
        return {k: v for op in self.optimizers for k, v in op.state.items()}

    def state_dict(self):
        """ ? """
        return [op.state_dict() for op in self.optimizers]

    def load_state_dict(self, state_dicts):
        """ ? """
        assert len(state_dicts) == len(self.optimizers)
        for i in range(len(state_dicts)):
            self.optimizers[i].load_state_dict(state_dicts[i])


class Optimizer(object):
    """
    Controller class for optimization. Mostly a thin
    wrapper for `optim`, but also useful for implementing
    rate scheduling beyond what is currently available.
    Also implements necessary methods for training RNNs such
    as grad manipulations.

    Args:
      method (:obj:`str`): one of [sgd, adagrad, adadelta, adam]
      lr (float): learning rate
      lr_decay (float, optional): learning rate decay multiplier
      start_decay_steps (int, optional): step to start learning rate decay
      beta1, beta2 (float, optional): parameters for adam
      adagrad_accum (float, optional): initialization parameter for adagrad
      decay_method (str, option): custom decay options
      warmup_steps (int, option): parameter for `noam` decay

    We use the default parameters for Adam that are suggested by
    the original paper https://arxiv.org/pdf/1412.6980.pdf
    These values are also used by other established implementations,
    e.g. https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer
    https://keras.io/optimizers/
    Recently there are slightly different values used in the paper
    "Attention is all you need"
    https://arxiv.org/pdf/1706.03762.pdf, particularly the value beta2=0.98
    was used there however, beta2=0.999 is still arguably the more
    established value, so we use that here as well
    """

    def __init__(self, method, learning_rate, max_grad_norm,
                 lr_decay=1, start_decay_steps=None, decay_steps=None,
                 beta1=0.9, beta2=0.999,
                 adagrad_accum=0.0,
                 decay_method=None,
                 warmup_steps=4000
                 ):
        self.last_ppl = None
        self.learning_rate = learning_rate
        self.original_lr = learning_rate
        self.max_grad_norm = max_grad_norm
        self.method = method
        self.lr_decay = lr_decay
        self.start_decay_steps = start_decay_steps
        self.decay_steps = decay_steps
        self.start_decay = False
        self._step = 0
        self.betas = [beta1, beta2]
        self.adagrad_accum = adagrad_accum
        self.decay_method = decay_method
        self.warmup_steps = warmup_steps

    def set_parameters(self, params):
        """ ? """
        self.params = []
        self.sparse_params = []
        for k, p in params:
            if p.requires_grad:
                if self.method != 'sparseadam' or "embed" not in k:
                    self.params.append(p)
                else:
                    self.sparse_params.append(p)
        
        
        self.optimizer = optim.Adam(self.params, lr=self.learning_rate,
                                        betas=self.betas, eps=1e-9)

    def _set_rate(self, learning_rate):
        self.learning_rate = learning_rate
        if self.method != 'sparseadam':
            self.optimizer.param_groups[0]['lr'] = self.learning_rate
        else:
            for op in self.optimizer.optimizers:
                op.param_groups[0]['lr'] = self.learning_rate

    def step(self):
        """Update the model parameters based on current gradients.

        Optionally, will employ gradient modification or update learning
        rate.
        """
        self._step += 1

        # Decay method used in tensor2tensor.
        if self.decay_method == "noam":
            self._set_rate(
                self.original_lr *

                 min(self._step ** (-0.5),
                     self._step * self.warmup_steps**(-1.5)))

            # self._set_rate(self.original_lr *self.model_size ** (-0.5) *min(1.0, self._step / self.warmup_steps)*max(self._step, self.warmup_steps)**(-0.5))
        # Decay based on start_decay_steps every decay_steps
        else:
            if ((self.start_decay_steps is not None) and (
                     self._step >= self.start_decay_steps)):
                self.start_decay = True
            if self.start_decay:
                if ((self._step - self.start_decay_steps)
                   % self.decay_steps == 0):
                    self.learning_rate = self.learning_rate * self.lr_decay

        if self.method != 'sparseadam':
            self.optimizer.param_groups[0]['lr'] = self.learning_rate

        if self.max_grad_norm:
            clip_grad_norm_(self.params, self.max_grad_norm)
        self.optimizer.step()


In [ ]:
import torch
import torch.nn as nn
from pytorch_pretrained_bert import BertModel, BertConfig
from torch.nn.init import xavier_uniform_

# from models.encoder import TransformerInterEncoder, Classifier, RNNEncoder
# from models.optimizers import Optimizer

def build_optim(train_from, model, checkpoint):
    """ Build optimizer """
    saved_optimizer_state_dict = None

    if train_from != '':
        optim = checkpoint['optim']
        saved_optimizer_state_dict = optim.optimizer.state_dict()
    else:
        print(456)
        optim = Optimizer(
            'adam', 2e-3, 0,
            beta1= 0.9, beta2=0.999,
            decay_method='noam',
            warmup_steps= 8000)

    optim.set_parameters(list(model.named_parameters()))

    if train_from != '':
        optim.optimizer.load_state_dict(saved_optimizer_state_dict)
        # if args.visible_gpus != '-1':
        #     for state in optim.optimizer.state.values():
        #         for k, v in state.items():
        #             if torch.is_tensor(v):
        #                 state[k] = v.cuda()

        if (optim.method == 'adam') and (len(optim.optimizer.state) < 1):
            raise RuntimeError(
                "Error: loaded Adam optimizer from existing model" +
                " but optimizer state is empty")

    return optim
# def build_optim(model, checkpoint):
#     """ Build optimizer """

#     optim = Optimizer(
#         'adam', 2e-3, 0,
#         beta1= 0.9, beta2=0.999,
#         decay_method='noam',
#         warmup_steps= 8000)

#     optim.set_parameters(list(model.named_parameters()))


#     # if (optim.method == 'adam') and (len(optim.optimizer.state) < 1):
#     #     raise RuntimeError(
#     #         "Error: loaded Adam optimizer from existing model" +
#     #         " but optimizer state is empty")

#     return optim

class Classifier(nn.Module):
    def __init__(self, hidden_size):
        super(Classifier, self).__init__()
        self.linear1 = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, mask_cls):
        h = self.linear1(x).squeeze(-1)
        sent_scores = self.sigmoid(h) * mask_cls.float()
        return sent_scores
class Bert(nn.Module):
    def __init__(self, temp_dir, load_pretrained_bert, bert_config):
        super(Bert, self).__init__()
        if(load_pretrained_bert):
            self.model = BertModel.from_pretrained('bert-base-uncased', cache_dir=temp_dir)
        else:
            self.model = BertModel(bert_config)

    def forward(self, x, segs, mask):
        encoded_layers, _ = self.model(x, segs, attention_mask =mask)
        top_vec = encoded_layers[-1]
        return top_vec


temp_dir = '/content/drive/MyDrive/Pro2/temp'
class Summarizer(nn.Module):
    def __init__(self,device, load_pretrained_bert = False, bert_config = None):
        super(Summarizer, self).__init__()
        self.bert = Bert(temp_dir, load_pretrained_bert, bert_config)
        
        self.encoder = Classifier(self.bert.model.config.hidden_size)
        

      
        for p in self.encoder.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)

        self.to(device)
    def load_cp(self, pt):
        self.load_state_dict(pt, strict=True)

    def forward(self, x, segs, clss, mask, mask_cls, sentence_range=None):

        top_vec = self.bert(x, segs, mask)
        sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss]
        sents_vec = sents_vec * mask_cls[:, :, None].float()
        sent_scores = self.encoder(sents_vec, mask_cls).squeeze(-1)
        return sent_scores, mask_cls

In [ ]:
!pip install tensorboardX

     |████████████████████████████████| 122kB 4.9MB/s 


In [ ]:
from __future__ import division

import argparse
import glob
import os
import random
import signal
import time

import torch
from pytorch_pretrained_bert import BertConfig

import distributed
import torch_xla.utils.serialization as xser
import torch_xla.core.xla_model as xm



def train(device,train_from):

    model = Summarizer(device, load_pretrained_bert=True)
    
    if train_from != '':
        checkpoint = xser.load(train_from)
        model.load_cp(checkpoint)
        # optim = build_optim(train_from, model, checkpoint)
        checkpoint = 0
    # else:
    #     optim = build_optim(train_from,model, None)
 
    return model
    # model,optim


In [ ]:
def convert(datasets):
    def _pad(data, pad_id, width=-1):
        if (width == -1):
            width = max(len(d) for d in data)
        rtn_data = [d + [pad_id] * (width - len(d)) for d in data]
        return rtn_data
    pre_src = [x['src'] for x in datasets[:]]
    pre_labels = [x['labels'] for x in datasets[:]]
    pre_segs = [x['segs'] for x in datasets[:]]
    pre_clss = [x['clss'] for x in datasets[:]]
    # src_txt = [x['src_txt'] for x in datasets[:]]
    # tgt_txt = [x['tgt_txt'] for x in datasets[:]]

    src = torch.tensor(_pad(pre_src, 0))

    labels = torch.tensor(_pad(pre_labels, 0))
    segs = torch.tensor(_pad(pre_segs, 0))
    mask = ~(src == 0)

    clss = torch.tensor(_pad(pre_clss, -1))
    mask_cls = ~ (clss == -1)
    clss[clss == -1] = 0



    return src, labels,segs,clss,mask,mask_cls

In [ ]:
datasets = torch.load('/content/drive/MyDrive/Pro2/databert')

Tách dữ liệu train và test

In [ ]:
import random
random.shuffle(datasets)

In [1]:
datasets_test = datasets[:50000]

In [ ]:
torch.save(datasets_test,'/content/drive/MyDrive/Pro2/databert_train')

In [ ]:
torch.save(datasets[50000:],'/content/drive/MyDrive/Pro2/databert_test')

In [ ]:
datasets_test = torch.load('/content/drive/MyDrive/Pro2/databert_train')

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
src, labels,segs,clss,mask,mask_cls = convert(datasets_test)

In [ ]:
lenData = len(datasets_test)

In [ ]:
batch_size = 10
train_data = TensorDataset(src, labels,segs,clss,mask,mask_cls)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
 def timeSince(since):
        now = time.time()
        s = now - since
        m = math.floor(s / 60)
        s -= m * 60
        return '%dm %ds' % (m, s)


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    print(pred_flat.shape)
    print(labels_flat.shape)
    
    F1_score = f1_score(pred_flat, labels_flat, average='macro')
    
    return accuracy_score(pred_flat, labels_flat), F1_score


In [ ]:
from tqdm import tqdm_notebook

In [ ]:
from tqdm import tqdm_notebook
import torch_xla.utils.serialization as xser
path ='/content/drive/MyDrive/Pro2/WeightBert/bertWeight1'
checkpoint = xser.load(path)

In [ ]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
from tqdm import tqdm_notebook
# import torch_xla.core.xla_model as xm
import torch_xla.core.xla_model as xm
import torch_xla.utils.serialization as xser


import torch_xla.distributed.parallel_loader as pl

In [ ]:
device = xm.xla_device()
model = train(device,'')
model.load_cp(checkpoint)
fn_loss = torch.nn.BCELoss(reduction='sum')

In [ ]:
  #  xm.save(model.state_dict(), path)

# Train model

In [ ]:
params = []
for k, p in list(model.named_parameters()):
      if p.requires_grad:
          params.append(p)

bert_optim = optim.Adam(params, lr=2e-3, eps=1e-9)
bert_optim.param_groups[0]['lr'] = 2e-3
_step = 25000
warmup_steps= 4000
params = 0
bert_optim.param_groups[0]['lr'] = (2e-3) * min(_step ** (-0.5), _step * warmup_steps**(-1.5))


# model = model.to(device)
fn_loss = torch.nn.BCELoss(reduction='sum')


# _trainEpoch(device,model,bert_optim,fn_loss)
all_loss = []
epochs = 30
print_every = 20
plot_every = 20
dev = device


start = time.time()
res = 0
rows = 10



curr_loss = 0

minLoss = 10e9

for epoch in range(epochs):
    print('epoch: ',epoch)
    # x = 0

    para_train_loader = pl.ParallelLoader(train_dataloader, [device]).per_device_loader(device)
    for i, batch in tqdm_notebook(enumerate(para_train_loader)):

        model.zero_grad()
        bert_optim.zero_grad()

        src = batch[0].to(dev)
        labels = batch[1].to(dev)
        segs = batch[2].to(dev)
        clss = batch[3].to(dev)
        mask = batch[4].to(dev)
        mask_cls = batch[5].to(dev)



        sent_scores, mask = model(src, segs, clss, mask, mask_cls)

        

        loss = fn_loss(sent_scores, labels.float())
        loss = (loss*mask.float()).sum()



        (loss/loss.numel()).backward()
      
        # bert_optim.step()
        # if i % 150 ==0:
        #     _step += 1
        
        xm.optimizer_step(bert_optim, barrier=True)
        # xm.mark_step()

        curr_loss+= loss.item()
        print(loss.item())


    all_loss.append(curr_loss/rows)
    print(curr_loss/rows)

    curr_loss = 0
    # xm.save(model.state_dict(), path)

In [ ]:
device = xm.xla_device()
model = train(device,'')
model.load_cp(checkpoint)
# model = model.to(device)
params = []
for k, p in list(model.named_parameters()):
      if p.requires_grad:
          params.append(p)

bert_optim = optim.Adam(params, lr=2e-3, eps=1e-9)
bert_optim.param_groups[0]['lr'] = 2e-3
_step = 25000
warmup_steps= 4000
params = 0


# model = model.to(device)
fn_loss = torch.nn.BCELoss(reduction='sum')


# _trainEpoch(device,model,bert_optim,fn_loss)
all_loss = []
epochs = 500
print_every = 20
plot_every = 20
dev = device


start = time.time()
res = 0
rows = 20



curr_loss = 0

for epoch in range(epochs):
    print('epoch: ',epoch)
    # x = 0

    para_train_loader = pl.ParallelLoader(train_dataloader, [device]).per_device_loader(device)
    for i, batch in tqdm_notebook(enumerate(para_train_loader)):

        model.zero_grad()
        bert_optim.zero_grad()

        src = batch[0].to(dev)
        labels = batch[1].to(dev)
        segs = batch[2].to(dev)
        clss = batch[3].to(dev)
        mask = batch[4].to(dev)
        mask_cls = batch[5].to(dev)



        sent_scores, mask = model(src, segs, clss, mask, mask_cls)

        

        loss = fn_loss(sent_scores, labels.float())
        loss = (loss*mask.float()).sum()


        (loss/loss.numel()).backward()
      
        # bert_optim.step()
        if i % 150 ==0:
            _step += 1
        bert_optim.param_groups[0]['lr'] = (2e-3) * min(_step ** (-0.5), _step * warmup_steps**(-1.5))
        xm.optimizer_step(bert_optim, barrier=True)
        # xm.mark_step()

        curr_loss+= loss.item()


    all_loss.append(curr_loss/rows)
    print(curr_loss/rows)
    curr_loss = 0
    # xm.save(model.state_dict(), path)

# Train song song trên 8 TPU

In [ ]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
from tqdm import tqdm_notebook
# import torch_xla.core.xla_model as xm
import torch_xla.core.xla_model as xm
import torch_xla.utils.serialization as xser


import torch_xla.distributed.parallel_loader as pl
# path ='/content/drive/MyDrive/Pro2/WeightBert/bertWeight'

def simple_map_fn(index, flags):
    print(1)
    device = xm.xla_device()
    model = train(device,'')
    model.load_cp(checkpoint)
    # model = model.to(device)
    params = []
    for k, p in list(model.named_parameters()):
            if p.requires_grad:
                params.append(p)

    bert_optim = optim.Adam(params, lr=2e-3, eps=1e-9)
    bert_optim.param_groups[0]['lr'] = 2e-3
    _step = 1
    warmup_steps=4000
    params = 0


    # model = model.to(device)
    fn_loss = torch.nn.BCELoss(reduction='sum')
    

    # _trainEpoch(device,model,bert_optim,fn_loss)
    all_loss = []
    epochs = 500
    print_every = 20
    plot_every = 20
    dev = device


    start = time.time()
    res = 0
    rows = lenData



    curr_loss = 0

    for epoch in range(epochs):
        print('epoch: ',epoch)
        # x = 0

        para_train_loader = pl.ParallelLoader(train_dataloader, [device]).per_device_loader(device)
        for i, batch in tqdm_notebook(enumerate(para_train_loader)):

            model.zero_grad()
            bert_optim.zero_grad()

            src = batch[0].to(dev)
            labels = batch[1].to(dev)
            segs = batch[2].to(dev)
            clss = batch[3].to(dev)
            mask = batch[4].to(dev)
            mask_cls = batch[5].to(dev)



            sent_scores, mask = model(src, segs, clss, mask, mask_cls)

            

            loss = fn_loss(sent_scores, labels.float())
            loss = (loss*mask.float()).sum()


            (loss/loss.numel()).backward()
          
            # bert_optim.step()
            _step += 1
            bert_optim.param_groups[0]['lr'] = (2e-3) * min(_step ** (-0.5), _step * warmup_steps**(-1.5))
            xm.optimizer_step(bert_optim, barrier=True)
            # xm.mark_step()

            curr_loss+= loss.item()
      

        all_loss.append(curr_loss/rows)
        print(curr_loss/rows)


        curr_loss = 0
        xm.save(model.state_dict(), path)
    print("Process", index ,"is using", xm.xla_real_devices([str(device)])[0])
    xm.save(model.state_dict(), path) #save model



    # checkpoint = {
    #         'model': model.state_dict(),
    #         'optim': bert_optim,
    # }
    # print('master1')
    
    # print(1)
    # xm.save(bert_optim, path + '1')
    xm.rendezvous('init')


flags = {}
# Note: Colab only supports start_method='fork'
xmp.spawn(simple_map_fn, args=(flags,), nprocs=8, start_method='fork')


# Đánh giá hiệu suất

In [ ]:
from tqdm import tqdm_notebook
import torch_xla.utils.serialization as xser
path ='/content/drive/MyDrive/Pro2/WeightBert/bertWeight1'
checkpoint = xser.load(path)


In [ ]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
from tqdm import tqdm_notebook
# import torch_xla.core.xla_model as xm
import torch_xla.core.xla_model as xm
import torch_xla.utils.serialization as xser


import torch_xla.distributed.parallel_loader as pl

In [ ]:
device = xm.xla_device()
model = train(device,'')
model.load_cp(checkpoint)
fn_loss = torch.nn.BCELoss(reduction='sum')

In [ ]:
def convert1(datasets):
    def _pad(data, pad_id, width=-1):
        if (width == -1):
            width = max(len(d) for d in data)
        rtn_data = [d + [pad_id] * (width - len(d)) for d in data]
        return rtn_data
    pre_src = [x['src'] for x in datasets[:]]
    pre_labels = [x['labels'] for x in datasets[:]]
    pre_segs = [x['segs'] for x in datasets[:]]
    pre_clss = [x['clss'] for x in datasets[:]]
    src_txt = [x['src_txt'] for x in datasets[:]]
    tgt_txt = [x['tgt_txt'] for x in datasets[:]]

    src = torch.tensor(_pad(pre_src, 0))

    labels = torch.tensor(_pad(pre_labels, 0))
    segs = torch.tensor(_pad(pre_segs, 0))
    mask = ~(src == 0)

    clss = torch.tensor(_pad(pre_clss, -1))
    mask_cls = ~ (clss == -1)
    clss[clss == -1] = 0



    return src, labels,segs,clss,mask,mask_cls,src_txt,tgt_txt


In [ ]:
datasets_test = torch.load('/content/drive/MyDrive/Pro2/databert_test')
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
src, labels,segs,clss,mask,mask_cls, src_txt, tgt_txt = convert1(datasets_test[0:25000])

In [ ]:
model.load_cp(checkpoint)

In [ ]:
batch_size = 100
train_data = TensorDataset(src, labels,segs,clss,mask,mask_cls)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
can_path = '/content/drive/MyDrive/Pro2/ResultData/candidate'
gold_path='/content/drive/MyDrive/Pro2/ResultData/gold'

In [ ]:
import numpy as np
def _get_ngrams(n, text):
            ngram_set = set()
            text_length = len(text)
            max_index_ngram_start = text_length - n
            for i in range(max_index_ngram_start + 1):
                ngram_set.add(tuple(text[i:i + n]))
            return ngram_set

def _block_tri(c, p):
    tri_c = _get_ngrams(3, c.split())
    for s in p:
        tri_s = _get_ngrams(3, s.split())
        if len(tri_c.intersection(tri_s))>0:
            return True
    return False
list_selected_ids = []
with torch.no_grad():
  for step, batch in tqdm_notebook(enumerate(train_dataloader)):

      
      gold = []
      pred = []
      src = batch[0].to(device)
      labels = batch[1].to(device)
      segs = batch[2].to(device)
      clss = batch[3].to(device)
      mask = batch[4].to(device)
      mask_cls = batch[5].to(device)
      sent_scores, mask = model(src, segs, clss, mask, mask_cls)
      
      
      loss = fn_loss(sent_scores, labels.float())
      loss = (loss * mask.float()).sum()
      # if step % 1000:
      #     print(loss.item()/100)

      # batch_stats = Statistics(float(loss.cpu().data.numpy()), len(labels))
      # stats.update(batch_stats)

      sent_scores = sent_scores + mask.float()

      sent_scores = sent_scores.cpu().data.numpy()
      selected_ids = np.argsort(-sent_scores, 1)
      list_selected_ids.append(selected_ids)
      xm.mark_step()



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [ ]:
list_out = list_selected_ids[0]
for i, idx in enumerate(list_selected_ids[1:]):
    print(list_out.shape)
    print(idx.shape)
    list_out = np.concatenate((list_out,idx),axis=0)
list_out.shape

(100, 47)
(100, 47)
(200, 47)
(100, 47)
(300, 47)
(100, 47)
(400, 47)
(100, 47)
(500, 47)
(100, 47)
(600, 47)
(100, 47)
(700, 47)
(100, 47)
(800, 47)
(100, 47)
(900, 47)
(100, 47)
(1000, 47)
(100, 47)
(1100, 47)
(100, 47)
(1200, 47)
(100, 47)
(1300, 47)
(100, 47)
(1400, 47)
(100, 47)
(1500, 47)
(100, 47)
(1600, 47)
(100, 47)
(1700, 47)
(100, 47)
(1800, 47)
(100, 47)
(1900, 47)
(100, 47)
(2000, 47)
(100, 47)
(2100, 47)
(100, 47)
(2200, 47)
(100, 47)
(2300, 47)
(100, 47)
(2400, 47)
(100, 47)
(2500, 47)
(100, 47)
(2600, 47)
(100, 47)
(2700, 47)
(100, 47)
(2800, 47)
(100, 47)
(2900, 47)
(100, 47)
(3000, 47)
(100, 47)
(3100, 47)
(100, 47)
(3200, 47)
(100, 47)
(3300, 47)
(100, 47)
(3400, 47)
(100, 47)
(3500, 47)
(100, 47)
(3600, 47)
(100, 47)
(3700, 47)
(100, 47)
(3800, 47)
(100, 47)
(3900, 47)
(100, 47)
(4000, 47)
(100, 47)
(4100, 47)
(100, 47)
(4200, 47)
(100, 47)
(4300, 47)
(100, 47)
(4400, 47)
(100, 47)
(4500, 47)
(100, 47)
(4600, 47)
(100, 47)
(4700, 47)
(100, 47)
(4800, 47)
(100, 47)
(

(25000, 47)

In [ ]:
can_path = '/content/drive/MyDrive/Pro2/ResultData/candidate'
gold_path='/content/drive/MyDrive/Pro2/ResultData/gold'
pred_keys = []
with open(can_path, 'w') as save_pred:
    with open(gold_path, 'w') as save_gold:
        for i, idx in enumerate(list_out):
            _pred = []
            newDict = dict()
            if(len(src_txt[i])==0):
                continue
            for j in list_out[i][:len(src_txt[i])]:
                # print(i,j)
                if(j>=len( src_txt[i])):
                    continue
                candidate = src_txt[i][j].strip()

                if(not _block_tri(candidate,_pred)):
                    _pred.append(candidate)
                    newDict.update({j:candidate})
                if len(_pred) == len(tgt_txt[i].split("<q>")):
                    break
                # else:
                #     _pred.append(candidate)
            pred.append(_pred)
            gold.append(tgt_txt[i].replace("<q>"," "))
            pred_keys.append(newDict)
        for i in range(len(gold)):
            save_gold.write(gold[i].strip() +'\n')

        for i in range(len(pred)):
            # idx = []
            # for x in pred_keys[i]:
            #     idx.append(x)
            # idx.sort()
            # str_out = ""
            # for x in idx:
            #     str_out += pred_keys[i][x] + ' '
            # save_pred.write(str_out.strip()+'\n')
            save_pred.write(' '.join(pred[i]).strip()+'\n')

        # break


In [ ]:
list_candidate = []
list_gold = []
for x in open(gold_path):
    list_gold.append(x.replace("\n",""))
for x in open(can_path):
    list_candidate.append(x.replace("\n",""))

# Sử dụng độ đo ROUGE để đánh giá hiệu suất

In [ ]:
!pip install rouge-score
!pip install rouge/requirements.txt
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL','rouge2'], use_stemmer=True)
scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')
scores

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL','rouge2'], use_stemmer=True)
# scores = scorer.score('The quick brown fox jumps over the lazy dog',
#                       'The quick brown dog jumps on the log.')

In [ ]:
import pandas as pd
data  = pd.read_csv('/content/drive/MyDrive/Pro2/wikiP1.csv')

In [ ]:
sum_rouge1 = sum_rougeL=sum_rouge2 = 0
for i in range(25000):
    if i % 1000==0:
            print(i)
    scores = scorer.score(list_candidate[i],
                      list_gold[i].replace("<q>"," "))
    if i % 1000 ==0:
        print( scores['rouge1'][2],scores['rougeL'][2],scores['rouge2'][2])
    sum_rouge1  += scores['rouge1'][2]
    sum_rougeL += scores['rougeL'][2]
    sum_rouge2 += scores['rouge2'][2]

19000
0.2417582417582418 0.14285714285714288 0.07777777777777779
20000
0.23140495867768593 0.11570247933884296 0.03361344537815126
21000
0.23963133640552992 0.1382488479262673 0.04651162790697674
22000
0.2413793103448276 0.13793103448275862 0.0
23000
0.23255813953488372 0.13953488372093023 0.09523809523809523
24000
0.24096385542168672 0.14457831325301204 0.07407407407407408


# Kết quả

In [ ]:
(sum_rouge1/25000*100,sum_rouge2/25000*100, sum_rougeL/25000*100)

(29.301154408172042, 8.038535401516198, 17.5276467862946)